In [ ]:
import nibabel as nib
import numpy as np
from pathlib import Path
import shutil

In [ ]:
CUBE_SIZE = 15
OFFSET = 10
ZEROS = np.zeros((CUBE_SIZE, CUBE_SIZE, CUBE_SIZE))


def modify(nib_layer, path):
    layer = nib_layer.get_fdata()

    x_count = int(layer.shape[0] / OFFSET)
    y_count = int(layer.shape[1] / OFFSET)
    z_count = int(layer.shape[2] / OFFSET)

    for x in range(x_count):
        for y in range(y_count):
            #for z in range(z_count):
            z = 7
            cube = layer[
                x * OFFSET:x * OFFSET + CUBE_SIZE,
                y * OFFSET:y * OFFSET + CUBE_SIZE,
                z * OFFSET:z * OFFSET + CUBE_SIZE,
            ]
            if np.sum(cube) == 0.0:
                continue

            copy = np.copy(layer)
            copy[
                x * OFFSET:x * OFFSET + CUBE_SIZE,
                y * OFFSET:y * OFFSET + CUBE_SIZE,
                z * OFFSET:z * OFFSET + CUBE_SIZE,
            ] = ZEROS

            image = nib.Nifti1Image(copy, affine=nib_layer.affine)
            nib.save(image, str(path / f'{x}-{y}-{z}.nii.gz'))

In [ ]:
def prepare(image):
    base_path = Path('images/') / image
    assert base_path.exists()
    for modality in ['t1', 't1ce', 't2', 'flair']:
        path = base_path / modality
        if path.exists():
            shutil.rmtree(path)
        path.mkdir()

        file_path = base_path / 'original' / f'{modality}.nii.gz'
        layer = nib.load(str(file_path))
        modify(layer, path)


prepare('Brats18_2013_17_1')